In [1]:
import re
from utils.myutils import load_object, save_object

In [2]:
file_path_owl = "dbpedia/dbpedia_2016-10.owl"

owl_file = open(file_path_owl, 'r', encoding='utf-8')

relations_valid = load_object('relations_valid')

unit_dict = dict()
unit_reverse_dict = dict()

In [3]:
regex_xml_tag = re.compile('<rdfs:label xml:lang="en">([^<]+)</rdfs:label>')

def get_descr(relation):
    search_term = 'rdf:about="' + relation + '"'
    owl_file.seek(0)
    for line in owl_file:
        if search_term in line:
            #Read next line
            label_line = owl_file.readline()
            #print(str(relation) + " found in line " + line)
            #print("Assuming range in line " + range_line)
            
            try:
                rdfs_label_tag = regex_xml_tag.search(label_line)[1]
            except TypeError:
                label_line = owl_file.readline()
                try:
                    rdfs_label_tag = regex_xml_tag.search(label_line)[1]
                    print("WARNING! Could not find in-line label for relation " + relation)
                    print("Found type in the next line. Check whether this is correct!")
                except TypeError:
                    print("ERROR! Could not find a type for relation " + relation)
                    continue
            
            try:
                regex_xml_tag.search(label_line)[2]
            except IndexError:
                label = rdfs_label_tag#[27:-13]
                #print(relation + " : " + label)
                
                return label
            print("ERROR! Multiple Findings! Second Finding: " + regex_xml_tag.search(label_line)[1])
    
    print("WARNING! Could not find a label for relation " + relation + "!")
    return None

In [4]:
regex_unit = re.compile('\(([^\(\)]+)\)$')
for relation in relations_valid:
    
    label = get_descr(relation)
    relation = '<' + relation + '>'
    
    if label is None:
        unit_dict[relation] = None
        continue
    
    if '(' in label and label[-1:] == ')':
        try:
            unit = regex_unit.search(label)[0][1:-1]
            print(relation + ": " + unit)
            unit_dict[relation] = unit
        except IndexError:
            unit_dict[relation] = None
            continue
    

<http://dbpedia.org/ontology/depth>: μ
<http://dbpedia.org/ontology/minimumDischarge>: m³/s
<http://dbpedia.org/ontology/pistonStroke>: μ
<http://dbpedia.org/ontology/acceleration>: s
<http://dbpedia.org/ontology/meanRadius>: μ
<http://dbpedia.org/ontology/cylinderBore>: μ
<http://dbpedia.org/ontology/railGauge>: μ
<http://dbpedia.org/ontology/fees>: $
<http://dbpedia.org/ontology/lowerEarthOrbitPayload>: g
<http://dbpedia.org/ontology/discharge>: m³/s
<http://dbpedia.org/ontology/networth>: $
<http://dbpedia.org/ontology/surfaceArea>: m2
<http://dbpedia.org/ontology/voltageOfElectrification>: V
<http://dbpedia.org/ontology/minimumTemperature>: K
<http://dbpedia.org/ontology/orbitalPeriod>: s
<http://dbpedia.org/ontology/assetUnderManagement>: $
<http://dbpedia.org/ontology/imageSize>: px
<http://dbpedia.org/ontology/hipSize>: μ
<http://dbpedia.org/ontology/torqueOutput>: Nm
<http://dbpedia.org/ontology/frequency>: Hz
<http://dbpedia.org/ontology/areaUrban>: m2
<http://dbpedia.org/onto

In [5]:
#Invert dict
for relation, unit in unit_dict.items():
    
    if unit is None:
        continue
    elif unit in unit_reverse_dict:
        unit_reverse_dict[unit].append(relation)
    else:
        unit_reverse_dict[unit] = [relation]

In [6]:
unit_reverse_dict

{'$': ['<http://dbpedia.org/ontology/fees>',
  '<http://dbpedia.org/ontology/networth>',
  '<http://dbpedia.org/ontology/assetUnderManagement>',
  '<http://dbpedia.org/ontology/projectBudgetFunding>',
  '<http://dbpedia.org/ontology/programCost>',
  '<http://dbpedia.org/ontology/gross>',
  '<http://dbpedia.org/ontology/toll>',
  '<http://dbpedia.org/ontology/grossDomesticProduct>',
  '<http://dbpedia.org/ontology/salary>',
  '<http://dbpedia.org/ontology/careerPrizeMoney>',
  '<http://dbpedia.org/ontology/projectBudgetTotal>',
  '<http://dbpedia.org/ontology/cost>',
  '<http://dbpedia.org/ontology/revenue>',
  '<http://dbpedia.org/ontology/budget>',
  '<http://dbpedia.org/ontology/tuition>',
  '<http://dbpedia.org/ontology/assets>',
  '<http://dbpedia.org/ontology/equity>',
  '<http://dbpedia.org/ontology/unitCost>',
  '<http://dbpedia.org/ontology/netIncome>',
  '<http://dbpedia.org/ontology/operatingIncome>',
  '<http://dbpedia.org/ontology/endowment>'],
 '/sqkm': ['<http://dbpedia.o

In [7]:
save_object(unit_dict, 'unit_dict')
save_object(unit_reverse_dict, 'unit_reverse_dict')